In [3]:
# Limit = 10
# for i in "top 5 Veg biryani restaurants near me?":
#     if (i.isdigit()):
#         if(int(i) <= Limit and int(i) > 0):
#             print(Limit,i)
#             Limit = i
#             print(Limit,i)
#         else:
#             print("Thank you for your request! I can recommend up to 10 restaurants, so here are the top 10 based on your request.\n")
# print(Limit)

In [4]:
import random
import json

import torch

%run Model.ipynb import NeuralNet
%run NLTK.ipynb import Tokenize,BagOfWord

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('Data.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "Model2.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "BOT"

def get_response(msg):
    sentence = Tokenize(msg)
    Limit = 10
    for i in sentence:
        if (i.isdigit()):
            if(int(i) <= Limit and int(i) > 0):
                Limit = i
            else:
                print("Thank you for your request! I can recommend up to 10 restaurants, so here are the top 10 based on your request.\n")
    X = BagOfWord(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    print("Prob : ",prob.item())
    # if prob.item() > 0:
    for intent in intents['Data']:
        if tag == intent["tag"]:
            responce = random.choice(intent['responses'])
            Out = ""
            if isinstance(responce, list):
                for i in range(int(Limit)+1):
                    Out += responce[i]
                    Out += "\n"
                return Out
            else:
                return responce
                # print(f"{bot_name}: {random.choice(intent['responses'])}")
    # else:
    #     return (f"{bot_name}: I do not understand...")

C:\Users\SAYEERAM\AppData\Local\Temp\ipykernel_11640\4072346747.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(FILE)
